In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.metrics import balanced_accuracy_score

import scipy
from dython.nominal import associations

# Ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load Data

In [2]:
IDS_MAPPING_FN = "./data/IDS_mapping.csv"
DIABETIC_FN = "./data/diabetic_data.csv"

In [3]:
# read files
mapping = pd.read_csv(IDS_MAPPING_FN, header=None)
df = pd.read_csv(DIABETIC_FN)

In [4]:
MEDIC_COLUMNS = df.columns[24:46].tolist()
MEDIC_COLUMNS_TAKE = ["take_" + med for med in MEDIC_COLUMNS]
PREVIOUS_HOSPITAL_ENCOUNTERS = ["number_outpatient", "number_inpatient", "number_emergency"]

Create dictionary of code-value mappings of `admission_type_id`, `discharge_disposition_id`, and `admission_source_id` using the mapping provided in the data-folder, and map integer values to string values for readability. 

In [5]:
admission_type_dict = {}
discharge_disposition_dict = {}
admission_source_dict = {}

list1 = []
for i, j in zip(mapping[0].values, mapping[1].values):
  if len(str(i))>3:
    feature_name = i
  elif len(str(i))!=3:
    if feature_name == 'admission_type_id':
      admission_type_dict[int(i)] = j
    elif feature_name == 'discharge_disposition_id':
      discharge_disposition_dict[int(i)] = j
    elif feature_name == 'admission_source_id':
      admission_source_dict[int(i)] = j


df['admission_type'] = df['admission_type_id'].apply(lambda x: admission_type_dict[x])
df['discharge_disposition'] = df['discharge_disposition_id'].apply(lambda x: discharge_disposition_dict[x])
df['admission_source'] = df['admission_source_id'].apply(lambda x: admission_source_dict[x])

In [6]:
print(f"Number of unique encounters: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

Number of unique encounters: 101766
Number of columns: 53


## Preprocessing

Some patients have many encounters (up to 40).

We only keep the first observation for each unique patient to treat them as i.i.d random variables.

We filter to only keep observations with `admission_type` $\in$ [Emergency, Urgent, Elective].

In [7]:
df = df.groupby("patient_nbr").first(skipna=False).reset_index()
df = df[df['admission_type'].isin(['Emergency', 'Urgent', 'Elective'])]
df = df[df['race'].isin(['Caucasian', 'AfricanAmerican', 'Hispanic', 'Asian'])]
print(f"Number of unique encounters after only keeping first encounter for each patient and filtering by admission_type : {df.shape[0]}")

Number of unique encounters after only keeping first encounter for each patient and filtering by admission_type : 60571


#### Feature Engineering: Diabetic Information

To select features as predictor variables we are only interested in certain information about the patients, but not the exact values. Thus we aggregate the information of sets of columns into new columns:


* Blood Glucose Tests

    * `max_glu_serum_flag`: Whether a max glucose serum test was done at the hospital : `max_glu_serum`


    * `A1C_flag`: Whether an AC1 test was performed to monitor blood glucose levels : `AC1result`

    
* Diabetic Medication Information: 

    * `change_dosage`: Whether there was any change ("Up" or "Down") in the diabetic prescription dosages as a result of the hospital visit.
        * *The column `change` indicates if there was a change in diabetic medications (either dosage or generic name). So if `change` is marked as changed but `change_dosage` is none, then there must have been a change in the generic name, i.e. the chemical name of a medicine.*


    * `change_medicine`: Whether there was prescribed any new diabetic medication i.e. a change in the medicament as a result of the hospital visit.


    * `num_diabetic_prescriptions`: How many diabetic prescriptions the patient ongoingly had at the time of hospital visit - a count of entries that are $\in \{\text{"Steady"}, \text{"Up"}, \text{"Down"}\}$ in `MEDIC_COLUMNS`

    * `take_<medicine_name>`: Whether the patient takes the <medicine_name> 


* Admitted in the hospital within the previous year

    * `prev_year_hospital`: Whether the patient had any admissions in the hospital during the past year. 

* Health Insurance / Coverage

    * `blue_cross`: Patient has private insurance
    * `medicaid`: Patient has medicaid
    * `medicare`: Patient has medicare
    * `self_payed`: Patient payed up front
  
* `discharge_disposition_id`: is aggregated into `home`, `transfer`, `unknown`, and `other`

* `Age` is aggregated into `[0-30)`, `[30-60)`, and `[60-100)`

* Readmitted
    * `readmitted_flag`: Whether the patient was readmitted or not based on the columns `readmitted`


In [8]:
#### Blood glucose Tests ####
df['max_glu_serum_flag'] = df['max_glu_serum'].notnull().astype(int) # Max Glucose Serum test flag
df['A1C_flag'] = df['A1Cresult'].notnull().astype(int) # AC1 test flag

#### Diabetic Medication Information ####
# Change in dosage if any of the diabetics prescriptions has entries "Up" or "Down"
df['change_dosage'] = df[MEDIC_COLUMNS].isin(['Up', 'Down']).any(axis=1).astype(int)
# Check if each entry is in the set ['Up', 'Down', 'Steady'] and sum all True entries for each row
df['num_diabetic_prescriptions'] = df[MEDIC_COLUMNS].apply(lambda col: np.isin(col, ['Up', 'Down', 'Steady'])).sum(axis=1).astype(int)
# Change in medicine is assumed to be the case when original change column = 1 but change in dosage = 0
df['change_medicine'] = np.where((df['change'] == 'Ch') & (df['change_dosage'] == 0), 1, 0)

# Whether the patient takes any of the medications
for med in MEDIC_COLUMNS:
    if len(np.unique(df[med])) == 1:
        MEDIC_COLUMNS_TAKE.remove("take_" + med) # some medications were taken by no patients
        continue
    name = "take_" + med
    df[name] = df[med].isin(["Down", "Steady", "Up"])


#### Hospital Encounters during preceding year ####
df['prev_year_hospital'] = (df[PREVIOUS_HOSPITAL_ENCOUNTERS] > 0).any(axis=1).astype(int)

#### Insurance Billing ####
df['blue_cross'] = np.where(df["payer_code"]=="BC", 1, 0)
df['medicare'] = np.where(df["payer_code"]=="MC", 1, 0)
df['medicaid'] = np.where(df["payer_code"]=="MD", 1, 0)
df['self_pay'] = np.where(df["payer_code"]=="SP", 1, 0)

#### Discharge disposition id aggregated ####
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([1, 6, 8, 13], "home")
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([2, 3, 4, 5, 9, 10, 14, 15, 16, 17, 22, 23, 24, 27, 28, 29], "transfer")
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([18, 25, 26], "unknown")
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([7, 11, 12, 19, 20, 21], "other")

#### Readmitted ####
# y_i
df['readmitted_flag'] = np.where(df['readmitted']=='NO', 0, 1) # Readmitted flag

#### Age aggregaed ####
df['age'] = df['age'].replace(['[0-10)', '[10-20)','[20-30)'], '[0-30)')
df['age'] = df['age'].replace(['[30-40)', '[40-50)','[50-60)'], '[30-60)')
df['age'] = df['age'].replace(['[60-70)', '[70-80)','[80-90)', '[90-100)'], '[60-100)')

## Define variables

In [9]:
FEATURES_NUM = ["time_in_hospital", "num_lab_procedures", "num_procedures", 'num_diabetic_prescriptions', 
                "num_medications", 'prev_year_hospital', "number_diagnoses"]

FEATURES_BIN = ['max_glu_serum_flag', 'A1C_flag', 'change_dosage', 'change_medicine', "blue_cross", "medicaid", "medicare","self_pay"]

FEATURES_CAT = ['discharge_disposition_id', "admission_type_id"]

FEATURES = FEATURES_NUM + FEATURES_BIN + FEATURES_CAT + MEDIC_COLUMNS_TAKE

PROTECTED_FEATURES = ['age','race', 'gender']
PATIENTS = ["patient_nbr"]  
TARGET = ["readmitted_flag"]

In [13]:
df = df[PATIENTS + FEATURES + PROTECTED_FEATURES + TARGET]

In [15]:
#One hot encode categorical features and protected features
df_one_hot = pd.get_dummies(df, prefix=None, prefix_sep='_', dummy_na=False, columns=FEATURES_CAT, drop_first=False)
df_one_hot = df_one_hot.drop(["discharge_disposition_id_unknown"], axis = 1)
FEATURES_ONE_HOT = df_one_hot.drop(PATIENTS+PROTECTED_FEATURES+TARGET, axis=1).columns.tolist()

df_one_hot = pd.get_dummies(df_one_hot, prefix=None, prefix_sep='_', dummy_na=False, columns=PROTECTED_FEATURES, drop_first=False)
PROTECTED_FEATURES_ONE_HOT = df_one_hot.drop(PATIENTS+FEATURES_ONE_HOT+TARGET, axis=1).columns.tolist()

Undersample majority class

In [19]:
df_major = df_one_hot[df_one_hot['readmitted_flag'] == 0]
df_minor = df_one_hot[df_one_hot['readmitted_flag'] == 1]

df_downsamples = resample(df_major, replace=False, n_samples=len(df_minor), random_state=42)  

df_one_hot = pd.concat([df_downsamples, df_minor])

Define train/test split

In [20]:
X = df_one_hot[FEATURES_ONE_HOT]
y = df_one_hot[TARGET].to_numpy().reshape(-1)
protected_features = df_one_hot[PROTECTED_FEATURES_ONE_HOT]

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(X, y, protected_features, test_size=0.2, random_state=42)

In [21]:
scaler = StandardScaler()
scaler.fit(X_train[FEATURES_NUM])
X_train[FEATURES_NUM] = scaler.transform(X_train[FEATURES_NUM])
X_test[FEATURES_NUM] = scaler.transform(X_test[FEATURES_NUM])

Using Grid search and Cross Validation we found an inverse regularisation strength of $0.1$  

In [104]:
def fit_logistic_regression(features_train, targets_train, features_test):
    logistic_model = LogisticRegression(max_iter=5000, penalty='l2', C=0.1, tol=1e-4, solver = "saga")
    logistic_model.fit(features_train, targets_train)
    y_pred = logistic_model.predict(features_test)
    y_proba = logistic_model.predict_proba(features_test)
    coefs = logistic_model.coef_
    return y_pred, y_proba, coefs

In [51]:
def get_metrics_overall(y_test_, y_pred_):
    accuracy = accuracy_score(y_test_, y_pred_)
    fpr, tpr, thresholds = roc_curve(y_test_, y_pred_)
    
    return accuracy, tpr[1], fpr[1]

In [120]:
def get_metrics_per_group(group_test_: pd.DataFrame(), y_test_: np.array(), y_pred_: np.array(), protected_group: str, protected_feature_names=PROTECTED_FEATURES_ONE_HOT):
    '''
    Calculates accuracy, true positive rate, (tpr), and false positive rate (fpr) for the specified protected group. 
    '''
    group_names, accuracies, tprs, fprs = [], [], [], []
    protected_feature_names = [group for group in protected_feature_names if protected_group in group]
    df_ = pd.concat([group_test_, pd.DataFrame({'y_test': y_test_, 'y_pred': y_pred_}, index=group_test_.index)], axis=1)
    
    for group_col in protected_feature_names:
        group_df = df_[df_[group_col]==True]
        y_true = group_df['y_test']
        y_pred = group_df['y_pred']
        
        accuracy = accuracy_score(y_true, y_pred)
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        
        group_names.append(group_col)
        accuracies.append(accuracy)
        tprs.append(tpr[1])
        fprs.append(fpr[1])
        
    return group_names, accuracies, tprs, fprs

## Decorrelate dataset

We define the function for decorrelating features using the method from the paper "A Geometric Solution to Fair Representations".

In [136]:
def correlation_remover(features, protected_features, lambda_):
    '''
    Decorrelate non-protected features with protected features.
    Lambda_ (fairness level) is in the interval [0,1], where 0; high fairness and 1; low fairness. 
    '''
    assert features.shape[0]==protected_features.shape[0]
    
    features_cols = features.columns
    
    #convert features to numeric matrices so we can do linalg operations
    features = features.astype(int).values
    protected_features = protected_features.astype(int).values
    
    # Find orthonormal basis of protected features
    orthobasis = scipy.linalg.orth(protected_features)
    
    # calculate projection of non-protected features onto orthonormal basis)
    features_decorrelated = features - orthobasis @ (orthobasis.T @ features)
    
    # Controlling the level of fairness. Subtract the projection from the original nonprotected features and scaling with fairness_level.
    features_decorrelated = features_decorrelated + lambda_ * (features - features_decorrelated)
    
    #convert to dataframe using the extracted column names
    features_decorrelated = pd.DataFrame(features_decorrelated, columns=features_cols)
    
    return features_decorrelated

In [127]:
overall_accuracies, tprs, fprs = [], [], []
gender_accuracies, gender_tprs, gender_fprs = [], [], []
race_accuracies, race_tprs, race_fprs = [], [], []
age_accuracies, age_tprs, age_fprs = [], [], []

num_lambda = 31
#for lambda_ in np.logspace(-(num_lambda-1), 0, num_lambda):
for lambda_ in np.linspace(0,1,num_lambda):
    X_train_decorrelated = correlation_remover(X_train, group_train, lambda_)
    X_test_decorrelated = correlation_remover(X_test, group_test, lambda_)
    
    predictions, _, _ = fit_logistic_regression(X_train_decorrelated, y_train, X_test_decorrelated)
    
    accuracy, tpr, fpr = get_metrics_overall(y_test, predictions)
    gender_name, gender_accuracy, gender_tpr, gender_fpr = get_metrics_per_group(group_test, y_test, predictions, 'gender')
    race_name, race_accuracy, race_tpr, race_fpr = get_metrics_per_group(group_test, y_test, predictions, 'race')
    age_name, age_accuracy, age_tpr, age_fpr = get_metrics_per_group(group_test, y_test, predictions, 'age')
        
    #Append to all lists
    overall_accuracies.append(accuracy)
    tprs.append(tpr)
    fprs.append(fpr)
    
    gender_accuracies.append(gender_accuracy)
    gender_tprs.append(gender_tpr)
    gender_fprs.append(gender_fpr)
    
    race_accuracies.append(race_accuracy)
    race_tprs.append(race_tpr)
    race_fprs.append(race_fpr)
    
    age_accuracies.append(age_accuracy)
    age_tprs.append(age_tpr)
    age_fprs.append(age_fpr)

## Figures


In [ ]:
df_one_hot_protected = pd.get_dummies(df[PROTECTED_FEATURES], prefix=None, prefix_sep='_', dummy_na=False, columns=PROTECTED_FEATURES, drop_first=False) 
corr_matrix = associations(pd.concat([df[FEATURES], df_one_hot_protected], axis=1), figsize=(20, 20), plot=False, num_num_assoc='pearson')
corr_matrix = corr_matrix['corr'][df_one_hot_protected.columns.tolist()].reset_index()
corr_matrix = corr_matrix.drop([i for i in range(len(FEATURES)-1,corr_matrix.shape[0])])
corr_matrix = corr_matrix.set_index('index')

In [139]:
# plt.figure(figsize=(6, 10))
# sns.heatmap(corr_matrix, annot=False, cmap='vlag', fmt=".1f", cbar=True)
# plt.show()